# Frozen Bot
Heladerías Frozen SRL nos solicita la construcción de un bot para la toma de
pedidos. Es nuestra responsabilidad el desarrollo de funciones auxiliares que le darán al bot la capacidad de desenvolverse en la conversación.

Flujo de diálogo

El siguiente diagrama ejemplifica el diálogo del bot:

![Flujo de diálogo](images/flujo.jpg)

Como se puede observar en el diagrama, en determinados momentos, el bot necesitará realizar validaciones externas para evaluar cómo continuar.

A continuación, se define el alcance de esas funciones y se solicita desarrollar las mismas.

## 1: GeoAPI

Completar el método is_hot_in_pehuajo con el siguiente objetivo:
* Consultar la información de clima y devolver True si la temperatura actual
supera los 28 grados celsius o False caso contrario. Esto implica incluso
devolver False ante cualquier excepción http.

API Información de clima: 
Link a documentacion: https://openweathermap.org/current#geo

Comenzamos importando requests para realizar solicitudes HTTP a la API de información climática. Definimos la clase GeoAPI y el método is_hot_in_pehuajo dentro de ella.

Nota: puedes instalar requests desde el propio notebook con :
%pip install requests ó !pip install requests

También voy a instalar logging para manejar exepciones y que no se vea afectado el retorno de la función.

%pip install logging

In [2]:
import logging
import requests

logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

class GeoAPI:
    """
    Clase para interactuar con la API de información de clima.
    """

    API_KEY = "d81015613923e3e435231f2740d5610b"
    LAT = "-35.836948753554054" 
    LON = "-61.870523905384076"
    
    MAX_TEMP = 28

    @classmethod
    def is_hot_in_pehuajo(cls):
        """
        Consulta la información de clima y devuelve True si la temperatura actual supera los 28 grados Celsius,
        o False en caso contrario. Devuelve False ante cualquier excepción http.

        :return: True si la temperatura es mayor a 28 grados, False en caso contrario.
        :rtype: bool
        """
        try:
            url = f"https://api.openweathermap.org/data/2.5/weather"
            parameters = {"lat": cls.LAT, "lon": cls.LON, "appid": cls.API_KEY, "units": "metric"}
            response = requests.get(url, parameters)

            if response.status_code == 200:
                data = response.json()
                temperature = data["main"]["temp"]

                # Registrar mensaje informativo
                logger.info(f"La temperatura actual en Pehuajó es {temperature} grados Celsius")

                return temperature > cls.MAX_TEMP
            else:
                # Registrar mensaje de error
                logger.error("No se pudo obtener la información del clima")

                return False
        except requests.exceptions.RequestException:
            # Registrar mensaje de excepción
            logger.exception("Ocurrió un error al realizar la solicitud HTTP")

            return False
        
# Llamar al método is_hot_in_pehuajo para verificar si hace calor en Pehuajó
result = GeoAPI.is_hot_in_pehuajo()

# Imprimir el resultado
if result:
    print("¡Hace calor en Pehuajó!")
else:
    print("No hace calor en Pehuajó.")


2023-07-09 00:03:13,271 - DEBUG - Starting new HTTPS connection (1): api.openweathermap.org:443
2023-07-09 00:03:14,545 - DEBUG - https://api.openweathermap.org:443 "GET /data/2.5/weather?lat=-35.836948753554054&lon=-61.870523905384076&appid=d81015613923e3e435231f2740d5610b&units=metric HTTP/1.1" 200 489
2023-07-09 00:03:14,549 - INFO - La temperatura actual en Pehuajó es 7 grados Celsius


No hace calor en Pehuajó.


Nota: Si estas utilizando Visual Estudio Code y si no lo hiciste antes, al ejecutar la primera celda con python te debería salir la ventana emrgente siguiente, donde debes instalar ipykernel package:

![instalar ipykernel package](images/ipykernel-package.jpg)

Por último realizamos las pruebas unitarias: 

* Para ello llevamos al código, el que estuvimos experimentando, a un archivo  [src/geoapi.py](src/geoapi.py)
* Instalamos pytest. %pip install pytest
* Y creamos un archivo [tests/test_geoapi.py](tests/test_geoapi.py)

El proyecto va tomando forma, la estructura nos va quedando, más o menos, asi:


![Estructura de archivos](images/estructura.jpg)

Ve al directorio raiz del proyecto y ejecuta: pytest ó bien pytest -v

"pytest -v" es muy útil cuando deseas una salida más detallada y legible durante la ejecución de las pruebas, lo que facilita la identificación de pruebas individuales y su resultado.

Ejemplo a continuación:

![Test is_hot_in_pehuajo](images/test_geoapi.jpg)

Para más detalle acerca del desarrollo de las pruebas ve al archivo [tests/test_geoapi.py](tests/test_geoapi.py). 

## 2: Función is_product_available

Dadas las variables: product_name y quantity, voy a completar la función is_product_available, función que va a buscar en un pandas DataFrame (simulando una base de datos) y devolver True si existe stock, False en caso contrario.

Nota: si miramos el diagrama de flujo al momento de la decisión de stock, encontramos un potencial loop infinito, ya que el usuario podria continuar ingresando productos invalidos o sin stock.

En primer lugar instalamos pandas si no lo tenemos aún:

%pip install pandas

In [6]:
import logging
import pandas as pd

_PRODUCT_DF = pd.DataFrame({
    "product_name": ["Chocolate", "Granizado", "Limon", "Dulce de Leche"], 
    "quantity": [3, 10, 0, 5]
})

# Configuración básica de logging para mostrar los mensajes en la consola
logging.basicConfig(level=logging.DEBUG,
                    format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

def is_product_available(product_name, quantity):
    """
    Verifica si un producto está disponible en el inventario.

    :param product_name: Nombre del producto a verificar.
    :type product_name: str
    :param quantity: Cantidad deseada del producto.
    :type quantity: int

    :return: True si el producto está disponible en la cantidad deseada, False en caso contrario.
    :rtype: bool
    """
    try:
        if quantity < 1:
            # Registrar mensaje de error
            logger.error("Producto no disponible")
            return False
        product = _PRODUCT_DF.loc[_PRODUCT_DF["product_name"].str.lower() == product_name.lower(), "quantity"]
        if not product.empty:
            stock = product.iloc[0]
            # Registrar mensaje informativo
            logger.info(f"Producto: {product_name}, Cantidad: {stock}, Disponible: {stock >= quantity}")
            return stock >= quantity
        else:
            # Registrar mensaje de error
            logger.error("Producto no disponible")
            return False
    except Exception as e:
        # Registrar mensaje de excepción
        logger.exception("Ocurrió un error al verificar la disponibilidad del producto: %s", str(e))
        return False

result = is_product_available("Dulce de leche", 2)

# Imprimir el resultado
if result:
    print("¡Hay stock!")
else:
    print("No hay...")

2023-07-09 21:46:01,054 - INFO - Producto: Dulce de leche, Cantidad: 5, Disponible: True


¡Hay stock!


Más ejemplos:

In [7]:
# Ejemplos de prueba
test_cases = [
    ("Chocolate", 2),  # Producto disponible en cantidad suficiente
    ("Granizado", 15),  # Producto disponible, pero cantidad insuficiente
    ("Limon", 0),  # Cantidad deseada es 0
    ("Dulce de Leche", 8),  # Producto disponible, pero cantidad insuficiente
    ("Mora", 4),  # Producto no existente
]

for product, quantity in test_cases:
    result = is_product_available(product, quantity)
    print(f"Producto: {product}, Cantidad: {quantity}, Disponible: {result}")

2023-07-09 21:56:51,961 - INFO - Producto: Chocolate, Cantidad: 3, Disponible: True
2023-07-09 21:56:51,963 - INFO - Producto: Granizado, Cantidad: 10, Disponible: False
2023-07-09 21:56:51,965 - ERROR - Producto no disponible
2023-07-09 21:56:51,966 - INFO - Producto: Dulce de Leche, Cantidad: 5, Disponible: False
2023-07-09 21:56:51,967 - ERROR - Producto no disponible


Producto: Chocolate, Cantidad: 2, Disponible: True
Producto: Granizado, Cantidad: 15, Disponible: False
Producto: Limon, Cantidad: 0, Disponible: False
Producto: Dulce de Leche, Cantidad: 8, Disponible: False
Producto: Mora, Cantidad: 4, Disponible: False


La función realiza una verificación inicial para determinar si la cantidad deseada es menor que 1. En ese caso, se registra un mensaje de error y se devuelve False. Esto asegura que si la cantidad es inválida (por ejemplo, un número negativo), la función no continúa con el proceso de búsqueda en el inventario y se sale de inmediato.

Además, si el producto no se encuentra en el DataFrame _PRODUCT_DF, se registra un mensaje de error y se devuelve False. No hay ningún bucle en la función que permita una iteración continua en busca del producto, por lo tanto no presenta riesgo de entrar en un bucle infinito.

Para las pruebas unitarias: 

* Creamos el archivo  [src/product_available.py](src/product_available.py)
* Y el archivo [tests/test_product_available.py](tests/test_product_available.py)

Luego para ejecutar pytest podemos hacerlo directamente sobre el archivo, por ejemplo de la siguiente manera:

pytest tests\test_product_available.py -v 

![Test product_available](images/test_available.jpg)

Más detalle acerca del desarrollo de las pruebas en al archivo [tests/test_product_available.py](tests/test_product_available.py). 

## 3: Función validate_discount_code

Completar la función validate_discount_code con el siguiente objetivo:

* Dada la lista de códigos de descuento vigentes y un código de descuento mencionado por el cliente, devuelve True si la diferencia entre el código mencionado y los códigos vigentes es menor a tres caracteres, en al menos
uno de los casos.

Nota: por diferencia se entiende caracteres que están presentes en el código brindado, pero
no en el código evaluado de la lista o viceversa (-desde ya esto me hace pensar en conjuntos-).

Ejemplos:

"primavera2021" deberia devolver True, ya que al compararlo:

vs "Primavera2021" = 2 caracteres de diferencia ("p" y "P")

vs "Verano2021" = 7 caracteres de diferencia ('i', 'n', 'o',
'm', 'V', 'p', 'v')

vs "Navidad2x1" = 8 caracteres de diferencia ('N', 'm', '0',
'x', 'e', 'd', 'p', 'r')

vs "heladoFrozen" = 14 caracteres de diferencia ('z', 'i',
'v', 'n', 'o', 'm', '2', '0', 'd', 'p', '1', 'F', 'h', 'l')

Confirmado! para calcular la diferencia de caracteres entre el código mencionado y el código vigente voy a utilizar el operador de diferencia asimétrica entre conjuntos.

In [3]:
import logging

# Configuración básica de logging para mostrar los mensajes en la consola
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

_AVAILABLE_DISCOUNT_CODES = ["Primavera2021", "Verano2021", "Navidad2x1", "heladoFrozen"]

def validate_discount_code(discount_code):
    for code in _AVAILABLE_DISCOUNT_CODES:
        difference = len(set(discount_code) ^ set(code))
        logger.debug(f"Diferencia entre '{discount_code}' y '{code}': {difference} caracteres")
        if difference < 3:
            return True
    return False

In [6]:
# Ejemplo 1: Diferencia menor a 3 caracteres
discount_code = "primavera2021"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 2: Diferencia mayor a 3 caracteres
discount_code = "Helado2021"
result = validate_discount_code(discount_code)
print(result)  # Output: False

# Ejemplo 3: Diferencia menor a 3 caracteres
discount_code = "Navidad2x2"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 4: Diferencia menor a 3 caracteres con mayúsculas y minúsculas
discount_code = "Verano2021"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 5: Diferencia mayor a 3 caracteres con mayúsculas y minúsculas
discount_code = "NAVIDAD2X1"
result = validate_discount_code(discount_code)
print(result)  # Output: False

2023-07-10 11:16:13,797 - DEBUG - Diferencia entre 'primavera2021' y 'Primavera2021': 2 caracteres
2023-07-10 11:16:13,799 - DEBUG - Diferencia entre 'Helado2021' y 'Primavera2021': 9 caracteres
2023-07-10 11:16:13,800 - DEBUG - Diferencia entre 'Helado2021' y 'Verano2021': 6 caracteres
2023-07-10 11:16:13,801 - DEBUG - Diferencia entre 'Helado2021' y 'Navidad2x1': 9 caracteres
2023-07-10 11:16:13,802 - DEBUG - Diferencia entre 'Helado2021' y 'heladoFrozen': 9 caracteres
2023-07-10 11:16:13,803 - DEBUG - Diferencia entre 'Navidad2x2' y 'Primavera2021': 9 caracteres
2023-07-10 11:16:13,805 - DEBUG - Diferencia entre 'Navidad2x2' y 'Verano2021': 12 caracteres
2023-07-10 11:16:13,806 - DEBUG - Diferencia entre 'Navidad2x2' y 'Navidad2x1': 1 caracteres
2023-07-10 11:16:13,807 - DEBUG - Diferencia entre 'Verano2021' y 'Primavera2021': 7 caracteres
2023-07-10 11:16:13,808 - DEBUG - Diferencia entre 'Verano2021' y 'Verano2021': 0 caracteres
2023-07-10 11:16:13,810 - DEBUG - Diferencia entre '

True
False
True
True
False


Si cambio el orden de "Primavera2021", lo coloco al final en el dataframe...

In [7]:
import logging

# Configuración básica de logging para mostrar los mensajes en la consola
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

_AVAILABLE_DISCOUNT_CODES = ["Verano2021", "Navidad2x1", "heladoFrozen", "Primavera2021"]

def validate_discount_code(discount_code):
    for code in _AVAILABLE_DISCOUNT_CODES:
        difference = len(set(discount_code) ^ set(code))
        logger.debug(f"Diferencia entre '{discount_code}' y '{code}': {difference} caracteres")
        if difference < 3:
            return True
    return False

Y pruebo:

In [8]:
# Ejemplo 1: Diferencia menor a 3 caracteres
discount_code = "primavera2021"
result = validate_discount_code(discount_code)
print(result)  # Output: True

2023-07-10 11:21:39,098 - DEBUG - Diferencia entre 'primavera2021' y 'Verano2021': 7 caracteres
2023-07-10 11:21:39,100 - DEBUG - Diferencia entre 'primavera2021' y 'Navidad2x1': 8 caracteres
2023-07-10 11:21:39,102 - DEBUG - Diferencia entre 'primavera2021' y 'heladoFrozen': 14 caracteres
2023-07-10 11:21:39,105 - DEBUG - Diferencia entre 'primavera2021' y 'Primavera2021': 2 caracteres


True


Obtengo el mismo resultado que el ejemplo mencionado al principio. Super!

Ahora vamos a optimizar el codigo utilizando manejo de exepciones, any() y list comprehension.

In [15]:
import logging

# Configuración básica de logging para mostrar los mensajes en la consola
logging.basicConfig(level=logging.DEBUG, format='%(asctime)s - %(levelname)s - %(message)s')
logger = logging.getLogger(__name__)

_AVAILABLE_DISCOUNT_CODES = ["Primavera2021", "Verano2021", "Navidad2x1", "heladoFrozen"]

def validate_discount_code(discount_code):
    """
    Valida si un código de descuento se encuentra vigente en la lista.

    :param discount_code: El código de descuento ingresado por el cliente.
    :type discount_code: str

    :return: True si la diferencia entre el código ingresado y los códigos vigentes es menor a tres caracteres
             en al menos uno de los casos. False en caso contrario o si se produce un error durante la validación.
    :rtype: bool
    """
    try:
        return any(len(set(discount_code) ^ set(code)) < 3 for code in _AVAILABLE_DISCOUNT_CODES)
    except Exception as e:
        logger.error(f"Error en la validación del código de descuento: {e}")
        return False


Utilizo la función any() junto con una list comprehension para verificar si existe al menos un código de descuento vigente en _AVAILABLE_DISCOUNT_CODES que cumpla con la condición de tener menos de 3 diferencias de caractares con el código dado.

Un bloque try-except para manejar cualquier excepción que pueda ocurrir durante la validación del código de descuento.

Devuelve True si se encuentra un código válido y False en caso de error o si no se encuentra ningún código válido.

In [16]:
# Ejemplo 1: Diferencia menor a 3 caracteres
discount_code = "primavera2021"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 2: Diferencia mayor a 3 caracteres
discount_code = "Helado2021"
result = validate_discount_code(discount_code)
print(result)  # Output: False

# Ejemplo 3: Diferencia menor a 3 caracteres
discount_code = "Navidad2x2"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 4: Diferencia menor a 3 caracteres con mayúsculas y minúsculas
discount_code = "Verano2021"
result = validate_discount_code(discount_code)
print(result)  # Output: True

# Ejemplo 5: Diferencia mayor a 3 caracteres con mayúsculas y minúsculas
discount_code = "NAVIDAD2X1"
result = validate_discount_code(discount_code)
print(result)  # Output: False

# Ejemplo 6: Código de descuento inválido (genera una excepción)
discount_code = 12345
result = validate_discount_code(discount_code)
print(str(result) + " -> Excepción")  # Output: False


2023-07-10 12:34:47,229 - ERROR - Error en la validación del código de descuento: 'int' object is not iterable


True
False
True
True
False
False -> Excepción
